<div class="alert alert-block alert-success"> <center> <h1> <b> Newsfeed Dashboard v3 </b> </h1> </center></div>

<div class="alert alert-block alert-danger"> <b> BEFORE RUNNING THIS NOTEBOOK, PLEASE MAKE SURE YOU INSTALLED CORE-NLP FOR SENTIMENT ANALYSIS.</b> </div>



**To Install CoreNLP, please follow the instructions on stack overflow: [Stanford NLP for Python](https://stackoverflow.com/questions/32879532/stanford-nlp-for-python). Note that you also need Java to be installed.** 

**Once the package is installed, you only need to compute these two following lines on your terminal each time you run this code:**

$ cd stanford-corenlp-full-2018-10-05

$ java -mx5g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -timeout 10000

**If any package is not loaded, try installing it using pip install** <i><u> package name </u></i>


### Import Libraries

In [19]:
import requests
from lxml import etree
import numpy as np
import pandas as pd
import datetime as dt
import json
import shapely
import shapely.geometry
import fiona
from pycorenlp import StanfordCoreNLP
import time
from newspaper import Article
from collections import Counter
from tqdm import tqdm
import ast
tqdm.pandas()

### Functions to get data from EMM

In [20]:
def datetorss(date_object):
    return date_object.strftime('%Y-%m-%d')+date_object.strftime('T%H')+'%3A'+date_object.strftime('%M')+'%3A'+date_object.strftime('%S')+'Z'

In [21]:
def get_data_cat_date(last_date,start_date,cat,countries):
    print('Running:',cat,'from:',start_date,'to:',last_date)

    original_title = list()
    links = list()
    description = list()
    pubdate = list()
    orig_lang = list()
    category = list()
    subcategory = list()
    location = list()
    trans_title = list()
    trans_descrip = list()
    entity = list()
    if countries == "":
        link = 'http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto='+datetorss(last_date)+'&datefrom='+datetorss(start_date)+'&category=UNDP_'+cat+'&datefrom='+start_date.strftime('%Y-%m-%d')+'&dateto='+last_date.strftime('%Y-%m-%d') 
    else:
        link = 'http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto='+datetorss(last_date)+'&datefrom='+datetorss(start_date)+'&category=UNDP_'+cat+'&datefrom='+start_date.strftime('%Y-%m-%d')+'&dateto='+last_date.strftime('%Y-%m-%d')+'&country='+countries 
    print(link)
    cont = requests.get(link)
    data_string = cont.content
    root = etree.XML(data_string)
    for element in root.iter('item'):
        original_title.append(element.find('title').text)
        links.append(element.find('link').text)
        description.append(element.find('description').text)
        pubdate.append(element.find('pubDate').text)
        orig_lang.append(element.find('{http://www.iso.org/3166}language').text)
        category.append(cat)
        u = []
        for j in element.findall('category'):
            if j.text != 'UNDP_'+cat:
                u.append(j.text)
        subcategory.append(u)
        try:
            location.append(element.find('{http://www.georss.org/georss}point').text)
        except:
            location.append(np.nan)
        try:
            trans_title.append(element.find('{http://emm.jrc.it}title').text)
            trans_descrip.append(element.find('{http://emm.jrc.it}description').text)
        except:
            trans_title.append(original_title[-1])
            trans_descrip.append(description[-1])
        m = []
        for child in element.findall('{http://emm.jrc.it}entity'):
            #print(child.attrib['name'])
            m.append(child.attrib['name'])
        entity.append(m)
    data = pd.DataFrame({'Original Title':original_title, 
                         'Link':links,
                         'Original Description':description,
                         'Publication Date':pubdate,
                         'Original Language':orig_lang,
                         'Category':category,
                         'Subcategory':subcategory,
                         'Location':location,
                         'Translated Title': trans_title,
                         'Translated Description': trans_descrip,
                         'Entity': entity})
    data['Publication Date'] = data['Publication Date'].apply(lambda date: dt.datetime.strptime(date.split(' ',1)[1],'%d %b %Y %H:%M:%S %z'))
    return data


In [22]:
def loop(cat,last_date,start_date,countries): 
    data = pd.DataFrame({'Original Title':[],
                                 'Link':[],
                                 'Original Description':[],
                                 'Publication Date':[],
                                 'Original Language':[],
                                 'Category':[],
                                 'Subcategory':[],
                                 'Location':[],
                                 'Translated Title': [],
                                 'Translated Description': [],
                                 'Entity': []})
    i = 0
    k = 0
    while start_date<last_date:
        len_data_prev = data.shape[0]
        data = pd.concat([data,get_data_cat_date(last_date,start_date,cat,countries)])
        len_data = data.shape[0]
        print('Loop number:',i+1,'--- Data size:',len(data))
        print('----------------------')
        if len(data) < 100:
            return data
        data['Publication Date'] = data['Publication Date'].apply(lambda x: x.replace(tzinfo=None))
        data.sort_values(by = 'Publication Date',ascending = False,inplace = True)
        data.index = range(len(data))
        last_date_2 = last_date
        last_date = data['Publication Date'].min().replace(tzinfo=None)
        if last_date_2 == last_date:
            if last_date - start_date >= dt.timedelta(days = 1):
                last_date = last_date - dt.timedelta(hours = np.random.normal(3*k,1))
                k += 1
            else:
                break
        if len_data_prev == len_data:
            break
        
        i+=1
    return data


In [23]:
def get_dataset(start_date,last_date,countries):
    categories = ['Conflict','Drought','Ecology','Genocide','HumanitarianAid','ManMadeDisasters','NaturalDisasters','PoliticalUnrest',
             'ScienceandTechnology','Security','Society','TerroristAttack','WaterConflict']
    data = pd.DataFrame({'Original Title':[],
                                 'Link':[],
                                 'Original Description':[],
                                 'Publication Date':[],
                                 'Original Language':[],
                                 'Category':[],
                                 'Subcategory':[],
                                 'Location':[],
                                 'Translated Title': [],
                                 'Translated Description': [],
                                 'Entity': []})
    for cat in categories:
        data = pd.concat([data, loop(cat,last_date,start_date,countries)])
    return data
        

### Get Full article

In [24]:
def get_article(link):
    try:
        article = Article(link)
        article.download()
        article.parse()
        return article.text.replace("\n"," ")
    except:
        return np.nan

## Processing the scraped dataset

### Get Article Relevance (Count of articles)

In [25]:
def get_relevant(data):
    df = data
    count = data.groupby('Translated Description').count()['Link']
    count_df = pd.DataFrame({'Translated Description': count.index, 'Count': count.values})
    df = df.drop_duplicates(subset = 'Translated Description').merge(count_df, on = 'Translated Description', how = 'left')
    return df


### Get Country

In [26]:
def distance(lat1,lon1,lat2,lon2):
    from math import sin, cos, sqrt, atan2, radians
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return(distance)

def get_country(lat, lon, borders,coord):
    try:
        closest = [(k,distance(lat,lon,coord[k][0],coord[k][1])) for k in coord.keys()]
        closest.sort(key=lambda elem: elem[1])
        # Find the associated biome
        for j in (closest):
            border_polygon = shapely.geometry.asShape(borders[j[0]]['geometry'])
            point = shapely.geometry.Point(lon, lat)
            if border_polygon.contains(point):
                country = borders[j[0]]['properties']['NAME']
                break
            else:
                country = np.nan
        return country
    except:
        return np.nan

def get_lat(x):
    try:
        return float((str(x)).split(' ')[0])
    except:
        return np.nan
    
def get_lon(x):
    try:
        return float((str(x)).split(' ')[1])
    except:
        return np.nan
        

def add_country(data,countries_borders, coord):
    data['Latitude'] = data['Location'].apply(lambda x: get_lat(x))
    data['Longitude'] = data['Location'].apply(lambda x: get_lon(x))
    data['Country'] = data.progress_apply(lambda x: get_country(x['Latitude'],x['Longitude'], countries_borders,coord), axis = 1)
    return data

<div class="alert alert-block alert-danger"> <center><b> THIS BLOCK IS TO DETERMINE THE COUNTRY CODE FOR A SPECIFIC COUNTRY.</b> </center></div>



In [27]:
country_dict = {'Afghanistan': 'AF',
 'Albania': 'AL',
 'Algeria': 'DZ',
 'Angola': 'AO',
 'Antigua and Barbuda': 'AG',
 'Argentina': 'AR',
 'Armenia': 'AM',
 'Aruba': 'AW',
 'Australia': 'AU',
 'Austria': 'AT',
 'Azerbaijan': 'AZ',
 'Bahamas': 'BS',
 'Bahrain': 'BH',
 'Bangladesh': 'BD',
 'Barbados': 'BB',
 'Belarus': 'BY',
 'Belgium': 'BE',
 'Belize': 'BZ',
 'Benin': 'BJ',
 'Bermuda': 'BM',
 'Bhutan': 'BT',
 'Bolivia': 'BO',
 'Bosnia': 'BA',
 'Botswana': 'BW',
 'Brazil': 'BR',
 'Brunei': 'BN',
 'Bulgaria': 'BG',
 'Burkina Faso': 'BF',
 'Burundi': 'BI',
 'Cambodia': 'KH',
 'Cameroon': 'CM',
 'Cape Verde': 'CV',
 'Cayman Islands': 'KY',
 'Central African Republic': 'CF',
 'China': 'CN',
 'Colombia': 'CO',
 'Comoros': 'KM',
 'Croatia': 'HR',
 'Cuba': 'CU',
 'Cyprus': 'CY',
 'Czech Republic': 'CZ',
 'Denmark': 'DK',
 'Djibouti': 'DJ',
 'Domenican Republic': 'DO',
 'Ecuador': 'EC',
 'Egypt': 'EG',
 'El Salvador': 'SV',
 'Ethiopia': 'ET',
 'Falkland Islands': 'FK',
 'Finland': 'FI',
 'France': 'FR',
 'French Guiana': 'GF',
 'Germany': 'DE',
 'Greece': 'GR',
 'Grenada': 'GD',
 'Guam': 'GU',
 'Guinea Bissau': 'GW',
 'Haiti': 'HT',
 'Honduras': 'HN',
 'Hong Kong': 'HK',
 'Hungary': 'HU',
 'Iceland': 'IS',
 'India': 'IN',
 'Indonesia': 'ID',
 'Iran': 'IR',
 'Iraq': 'IQ',
 'Ireland': 'IE',
 'Israel': 'IL',
 'Italy': 'IT',
 'Ivory Coast': 'CI',
 'Jamaica': 'JM',
 'Japan': 'JP',
 'Jordan': 'JO',
 'Kazakhstan': 'KZ',
 'Kenya': 'KE',
 'Kuwait': 'KW',
 'Kyrgyzstan': 'KG',
 'Laos': 'LA',
 'Latvia': 'LV',
 'Lebanon': 'LB',
 'Lesotho': 'LS',
 'Liberia': 'LR',
 'Libya': 'LY',
 'Liechtenstein': 'LI',
 'Lithuania': 'LT',
 'Luxemburg': 'LU',
 'Macau': 'MO',
 'Madagascar': 'MG',
 'Malawi': 'MW',
 'Malaysia': 'MY',
 'Maldives': 'MV',
 'Mali': 'ML',
 'Malta': 'MT',
 'Mauritania': 'MR',
 'Mauritius': 'MU',
 'Mexico': 'MX',
 'Moldova': 'MD',
 'Monaco': 'MC',
 'Mongolia': 'MN',
 'Monserrat': 'MS',
 'Montenegro': 'ME',
 'Morocco': 'MA',
 'Mozambique': 'MZ',
 'Myanmar': 'MM',
 'N/A': 'XK',
 'Namibia': 'NA',
 'Nepal': 'NP',
 'Netherlands': 'NL',
 'Netherlands Antilles': 'AN',
 'New Caledonia': 'NC',
 'New Zealand': 'NZ',
 'Nicaragua': 'NI',
 'Niger': 'NE',
 'Nigeria': 'NG',
 'North Korea': 'KP',
 'North Macedonia': 'MK',
 'Northern Marianas': 'MP',
 'Norway': 'NO',
 'Oman': 'OM',
 'Pakistan': 'PK',
 'Panama': 'PA',
 'Paraguay': 'PY',
 'Peru': 'PE',
 'Philippines': 'PH',
 'Poland': 'PL',
 'Porto Rico': 'PR',
 'Portugal': 'PT',
 'Qatar': 'QA',
 'Republic Congo': 'CG',
 'Reunion': 'RE',
 'Romania': 'RO',
 'Ruanda': 'RW',
 'Russia': 'RU',
 'Saint Kitts and Nevis': 'KN',
 'Saint Vincent and the Grenadines': 'VC',
 'Samoa': 'WS',
 'Saudi Arabia': 'SA',
 'Senegal': 'SN',
 'Serbia': 'RS',
 'Sierra Leone': 'SL',
 'Singapore': 'SG',
 'Slovakia': 'SK',
 'Slovenia': 'SI',
 'Somalia': 'SO',
 'South Africa': 'ZA',
 'South Korea': 'KR',
 'Spain': 'ES',
 'Sri Lanka': 'LK',
 'St. Lucia': 'LC',
 'Sudan': 'SD',
 'Surinam': 'SR',
 'Swaziland': 'SZ',
 'Sweden': 'SE',
 'Switzerland': 'CH',
 'Syria': 'SY',
 'Taiwan': 'TW',
 'Tajikistan': 'TJ',
 'Tanzania': 'TZ',
 'Thailand': 'TH',
 'Togo': 'TG',
 'Tonga': 'TO',
 'Trinidad and Tobago': 'TT',
 'Tunisia': 'TN',
 'Turkey': 'TR',
 'Turkmenistan': 'TM',
 'Tuvalu': 'TV',
 'USA': 'US',
 'Uganda': 'UG',
 'Ukraine': 'UA',
 'United Arab Emirates': 'AE',
 'United Kingdom': 'GB',
 'Uruguay': 'UY',
 'Uzbekistan': 'UZ',
 'Venezuela': 'VE',
 'Vietnam': 'VN',
 'Virgin Islands': 'VI',
 'West Sahara': 'EH',
 'Yemen': 'YE',
 'Zambia': 'ZM',
 'Zimbabwe': 'ZW'}



### Get Sentiment

In [28]:
def coreNLPanalysis(text,nlp):
    res = nlp.annotate(text,
                   properties={
                       'annotators': 'sentiment',
                       'outputFormat': 'json',
                       'timeout': 50000, 'ssplit.isOneSentence': True
                   })
    try:
        return res['sentences'][0]['sentimentDistribution'] 
    except:
        return [np.nan]*5
    
def arg(x,Full_text):
    if Full_text and x['Original Language'] == 'en':
        if not pd.isna(x["Full Article"]) and x["Full Article"] != "":
            return 'Full Article'
        else:
            return 'Translated Description'
    else:
        return 'Translated Description'
    
    
def get_sentiment(data,nlp,Full_text):
    data[['Very Neg', 'Negat', 'Neutral', 'Posit', 'Very Pos','On']] = data.progress_apply(lambda x: coreNLPanalysis(x[arg(x,Full_text)],nlp)+[arg(x,Full_text)],
                                                                                                   axis = 1, result_type = 'expand')
    return data



### Get Emotions

In [29]:
def get_emotion_dict():
    nrc = "./External_files/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt"
    count=0
    emotion_dict=dict()
    with open(nrc,'r') as f:
        all_lines = list()
        for line in f:
            if count < 46:
                count+=1
                continue
            line = line.strip().split('\t')
            if int(line[2]) == 1:
                if emotion_dict.get(line[0]):
                    emotion_dict[line[0]].append(line[1])
                else:
                    emotion_dict[line[0]] = [line[1]]
    return emotion_dict

def emotion_analyzer(text,emotion_dict=get_emotion_dict()):
    #Set up the result dictionary
    emotions = {x for y in emotion_dict.values() for x in y} 
    #list comprehension - emotion for (emotion_list in emotion_dict.values() for emotion in emotion_list
    emotion_count = dict()
    for emotion in emotions:
        emotion_count[emotion] = 0

    #Analyze the text and normalize by total number of words
    total_words = len(text.split())
    for word in text.split():
        if emotion_dict.get(word):
            for emotion in emotion_dict.get(word):
                emotion_count[emotion] += 1/total_words
    # Keys: Sadness, Joy, Surprise, Anticipation, Fear, Negative, Anger, Trust, Positive, Disgust
    return list(emotion_count.values())

def get_emotion(data):
    emotion_dict = get_emotion_dict()
    data[['Sadness', 'Joy', 'Surprise', 'Anticipation', 'Fear', 'Neg', 'Anger', 'Trust', 'Positive', 'Disgust']] = data.apply(lambda x: emotion_analyzer(x['Translated Description'],emotion_dict=emotion_dict),
                                                                                                   axis = 1, result_type = 'expand')
    data.drop(['Positive', 'Neg'], axis=1, inplace = True)
    #Columns: Sadness, Joy, Surprise, Anticipation, Fear, Negative, Anger, Trust, Positive, Disgust
    return data


### Get Most common entities

In [30]:
def get_list(x):
    if isinstance(x,str):
        return ast.literal_eval(x)
    else:
        return x


def get_most_common(data):
    data['Entity'] = data['Entity'].apply(lambda x: get_list(x))
    categories = ['Conflict','Drought','Ecology','Genocide','HumanitarianAid','ManMadeDisasters','NaturalDisasters','PoliticalUnrest',
             'ScienceandTechnology','Security','Society','TerroristAttack','WaterConflict']
    columns = ['Category']
    for i in range(20):
        columns.append('Most_common_'+str(i+1))
        columns.append('Count_'+str(i+1))

    data['Entity_all'] = data.apply(lambda x: x['Count']*x['Entity'],axis = 1)
    list_ = []   
    for cat in categories:
        df = data[data['Category']==cat]
        entities = sum(df['Entity_all'].as_matrix().flatten(), [])
        c = Counter(entities)
        small_list = list(sum(c.most_common(20),()))
        list_.append([cat]+small_list+(20-len(small_list)//2)*[np.nan,np.nan])
    
    list_ = np.array(list_)
    output = pd.DataFrame(list_,columns = columns)
    return output
        

### Final Function

In [31]:
def final_data():
    countries = input('Enter country codes (ex: US, FR, GB/all): ')
    while True:
        if countries != "all":
            l = [len(x) for x in countries.split(", ")]
            if set(l) != {2}:
                countries = input("-\nWrong entry format.\nEnter country codes (ex: US, FR, GB/all): ")
            else:
                break
        else:
            break
    if countries == 'all':
        countries = ""
    else:
        countries = countries.replace(', ','%2C')
    start_date = input("Enter the begining date (yyyy/mm/dd/hh/mm/ss): ")
    while True:
        try:
            start_date = dt.datetime.strptime(start_date,'%Y/%m/%d/%H/%M/%S')
            break
        except:
            start_date = input("-\nWrong entry format.\nEnter the begining date (yyyy/mm/dd/hh/mm/ss): ")
    last_date = input("Enter the ending date (yyyy/mm/dd/hh/mm/ss): ")
    while True:
        try:
            last_date = dt.datetime.strptime(last_date,'%Y/%m/%d/%H/%M/%S')
            break
        except:
            last_date = input("-\nWrong entry format.\nEnter the begining date (yyyy/mm/dd/hh/mm/ss): ")
    Full_text = input("Perform sentiment analysis on full text? (y/n): ")
    while True:
        if Full_text != 'y' and Full_text != 'n':
            Full_text = input("-\nWrong entry format.\nPerform sentiment analysis on full text? (y/n): ")
        else:
            break
    if Full_text == 'y':
        Full_text = True
    elif Full_text == 'n':
        Full_text = False
    data = get_dataset(start_date,last_date,countries)
    print('---------------------------------------------------')
    print('Running Relevance of data')
    data = get_relevant(data)
    print('---------------------------------------------------')
    print('Running countries')
    countries_borders = fiona.open('./External_files/Country_borders/Borders.shp')
    n = 246
    coord = dict()
    for i in range(n):
        lat = float(countries_borders[i]['properties']['Info de _1'])
        lon = float(countries_borders[i]['properties']['Info de gÃ'])
        coord[i] = (lat,lon)
    data = add_country(data,countries_borders,coord)
    if Full_text:
        print('---------------------------------------------------')
        print('Running full text')
        data['Full Article'] = data['Link'].progress_apply(lambda x: get_article(x))
        
    print('---------------------------------------------------')
    
    while True:
        try:
            nlp = StanfordCoreNLP('http://localhost:9000')
            print('Running Sentiment')
            print("Success")
            data = get_sentiment(data,nlp,Full_text)
            break
        except Exception as e :
            print(e)
            print('Running Sentiment')
            print('Unable. Please connect to CoreNLP server')
            time.sleep(20)
    print('---------------------------------------------------')
    print('Running Emotions')
        
    data = get_emotion(data)
    data.to_csv("Ukraine_UA_Jan_01_to_Apr_06.csv", index = False)
    data2 = get_most_common(data)
    return data, data2

In [32]:
#data_test = pd.read_csv("feb_march.csv")

#data_test = data_test.drop(columns = ['Count', 'Sadness', 'Joy', 'Surprise',
#       'Anticipation', 'Fear', 'Anger', 'Trust', 'Disgust', 'Entity_all'])


In [33]:
datab, data2b = final_data()

Enter country codes (ex: US, FR, GB/all): UA
Enter the begining date (yyyy/mm/dd/hh/mm/ss): 2019/01/01/00/00/00
Enter the ending date (yyyy/mm/dd/hh/mm/ss): 2019/04/06/23/59/00
Perform sentiment analysis on full text? (y/n): n
Running: Conflict from: 2019-01-01 00:00:00 to: 2019-04-06 23:59:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-04-06T23%3A59%3A00Z&datefrom=2019-01-01T00%3A00%3A00Z&category=UNDP_Conflict&datefrom=2019-01-01&dateto=2019-04-06&country=UA
Loop number: 1 --- Data size: 100
----------------------
Running: Conflict from: 2019-01-01 00:00:00 to: 2019-03-26 09:07:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-03-26T09%3A07%3A00Z&datefrom=2019-01-01T00%3A00%3A00Z&category=UNDP_Conflict&datefrom=2019-01-01&dateto=2019-03-26&country=UA
Loop number: 2 --- Data size: 200
----------------------
Running: Conflict from: 2019-01-01 00:00:00 to: 2019-03-15 09:29:00
http://emm.newsbrief.eu/rss/rss?langu

Loop number: 10 --- Data size: 1000
----------------------
Running: HumanitarianAid from: 2019-01-01 00:00:00 to: 2019-03-13 23:38:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-03-13T23%3A38%3A00Z&datefrom=2019-01-01T00%3A00%3A00Z&category=UNDP_HumanitarianAid&datefrom=2019-01-01&dateto=2019-03-13&country=UA
Loop number: 11 --- Data size: 1100
----------------------
Running: HumanitarianAid from: 2019-01-01 00:00:00 to: 2019-03-11 18:09:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-03-11T18%3A09%3A00Z&datefrom=2019-01-01T00%3A00%3A00Z&category=UNDP_HumanitarianAid&datefrom=2019-01-01&dateto=2019-03-11&country=UA
Loop number: 12 --- Data size: 1200
----------------------
Running: HumanitarianAid from: 2019-01-01 00:00:00 to: 2019-03-07 22:27:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-03-07T22%3A27%3A00Z&datefrom=2019-01-01T00%3A00%3A00Z&category=UNDP_HumanitarianAid

Loop number: 34 --- Data size: 3400
----------------------
Running: HumanitarianAid from: 2019-01-01 00:00:00 to: 2019-01-25 00:20:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-01-25T00%3A20%3A00Z&datefrom=2019-01-01T00%3A00%3A00Z&category=UNDP_HumanitarianAid&datefrom=2019-01-01&dateto=2019-01-25&country=UA
Loop number: 35 --- Data size: 3500
----------------------
Running: HumanitarianAid from: 2019-01-01 00:00:00 to: 2019-01-23 17:20:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-01-23T17%3A20%3A00Z&datefrom=2019-01-01T00%3A00%3A00Z&category=UNDP_HumanitarianAid&datefrom=2019-01-01&dateto=2019-01-23&country=UA
Loop number: 36 --- Data size: 3600
----------------------
Running: HumanitarianAid from: 2019-01-01 00:00:00 to: 2019-01-20 15:23:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-01-20T15%3A23%3A00Z&datefrom=2019-01-01T00%3A00%3A00Z&category=UNDP_HumanitarianAid

Loop number: 14 --- Data size: 1400
----------------------
Running: ManMadeDisasters from: 2019-01-01 00:00:00 to: 2019-03-24 13:47:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-03-24T13%3A47%3A00Z&datefrom=2019-01-01T00%3A00%3A00Z&category=UNDP_ManMadeDisasters&datefrom=2019-01-01&dateto=2019-03-24&country=UA
Loop number: 15 --- Data size: 1500
----------------------
Running: ManMadeDisasters from: 2019-01-01 00:00:00 to: 2019-03-23 10:38:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-03-23T10%3A38%3A00Z&datefrom=2019-01-01T00%3A00%3A00Z&category=UNDP_ManMadeDisasters&datefrom=2019-01-01&dateto=2019-03-23&country=UA
Loop number: 16 --- Data size: 1600
----------------------
Running: ManMadeDisasters from: 2019-01-01 00:00:00 to: 2019-03-22 11:26:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-03-22T11%3A26%3A00Z&datefrom=2019-01-01T00%3A00%3A00Z&category=UNDP_ManMadeDis

Loop number: 38 --- Data size: 3800
----------------------
Running: ManMadeDisasters from: 2019-01-01 00:00:00 to: 2019-02-26 15:13:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-02-26T15%3A13%3A00Z&datefrom=2019-01-01T00%3A00%3A00Z&category=UNDP_ManMadeDisasters&datefrom=2019-01-01&dateto=2019-02-26&country=UA
Loop number: 39 --- Data size: 3900
----------------------
Running: ManMadeDisasters from: 2019-01-01 00:00:00 to: 2019-02-26 10:31:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-02-26T10%3A31%3A00Z&datefrom=2019-01-01T00%3A00%3A00Z&category=UNDP_ManMadeDisasters&datefrom=2019-01-01&dateto=2019-02-26&country=UA
Loop number: 40 --- Data size: 4000
----------------------
Running: ManMadeDisasters from: 2019-01-01 00:00:00 to: 2019-02-25 15:57:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-02-25T15%3A57%3A00Z&datefrom=2019-01-01T00%3A00%3A00Z&category=UNDP_ManMadeDis

...

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [44]:
datab

,Original Title,Link,Original Description,Publication Date,Original Language,Category,Subcategory,Location,Translated Title,Translated Description,...,On,Sadness,Joy,Surprise,Anticipation,Fear,Anger,Trust,Disgust,Entity_all
0,министр иностранных дел,http://news.meta.ua/ua/metka:%D0%BC%D0%B8%D0%B...,и сроках выхода из Евросоюза. Как передает rep...,2019-04-06 17:09:00,ru,Conflict,"[Conflict, G7]",52.5163 13.3276,The minister of foreign affairs,The minister of foreign affairs,...,Translated Description,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"[Deutsche Welle, NATO, Carl Bildt, Heiko Maas,..."
1,Церковь разрушили бульдозерами и зарезали прих...,https://gazeta.ua/ru/articles/history/_cerkov-...,Геноцид в Руанде. Фото: blogforlife.org. Поздн...,2019-04-06 07:28:00,ru,Conflict,"[Conflict, UNbodies]",-1.95011 30.0588,The church destroyed by bulldozers and stabbed...,The church destroyed by bulldozers and stabbed...,...,Translated Description,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.000000,0.111111,"[Dassault Aviation, Juvenal Habyarimana, Dassa..."
2,Макрон хочет выяснить роль Франции в геноциде ...,https://nv.ua/world/geopolitics/makron-hochet-...,Во французскую комиссию войдут восемь историко...,2019-04-06 07:07:00,ru,Conflict,[Conflict],48.8521 2.34899,Makron wants to find out the role of France in...,Makron wants to find out the role of France in...,...,Translated Description,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,[]
3,Макрона приказал расследовать роль Франции в г...,https://www.pravda.com.ua/rus/news/2019/04/6/7...,Президент Франции Эммануэль Макрон поручил спе...,2019-04-06 05:48:00,ru,Conflict,[Conflict],48.8521 2.34899,Макрона ordered to investigate the role of Fra...,Макрона ordered to investigate the role of Fra...,...,Translated Description,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,[]
4,G7 опубликовала заявление по ситуации в Ливии,https://www.unn.com.ua/ru/news/1791483-g7-opub...,КИЕВ. 6 апреля. УНН. Министры иностранных дел ...,2019-04-06 02:57:00,ru,Conflict,"[FedericaMogherini, Conflict, UNbodies, G7]",52.5163 13.3276,G8 issued a statement on the situation in libya,G8 issued a statement on the situation in libya,...,Translated Description,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000,0.000000,0.000000,"[Federica Mogherini, G8, Federica Mogherini, G8]"
5,Совбез ООН требует прекратить военные действия...,https://www.unn.com.ua/ru/news/1791475-radbez-...,КИЕВ. 6 апреля. УНН. Совет безопасности ООН пр...,2019-04-06 01:13:00,ru,Conflict,"[Conflict, UNbodies]",52.5163 13.3276,Un security council demands to cease hostiliti...,Un security council demands to cease hostiliti...,...,Translated Description,0.111111,0.111111,0.111111,0.111111,0.000000,0.111111,0.000000,0.111111,"[Khalifa Haftar, Khalifa Haftar]"
6,"""Разминировать"" прошлое",https://zn.ua/international/razminirovat-prosh...,"Кто бы ни стал следующим президентом Украины, ...",2019-04-05 21:24:00,ru,Conflict,[Conflict],55.7388 37.6665,"""Clear"" past","""Clear"" past",...,Translated Description,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"[Prawo i Sprawiedliwość, NATO, Bronislaw Komor..."
7,Сьогодні влаштують облаву на водіїв,http://news.meta.ua/ua/cluster:64469045-Sogodn...,Про це повідомили в групі PITBULL KIEV в соціа...,2019-04-05 21:06:00,ru,Conflict,[Conflict],NaN,Сьогодні влаштують raided водіїв,Сьогодні влаштують raided водіїв,...,Translated Description,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,[Facebook]
8,Франция расследует свою причастность к геноцид...,https://news.liga.net/world/news/frantsiya-ras...,Президент Франции Эмманюэль Макрон создал спец...,2019-04-05 20:21:00,ru,Conflict,[Conflict],48.8521 2.34899,France is investigating his involvement in the...,France is investigating his involvement in the...,...,Translated Description,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [45]:
data2b

,Category,Most_common_1,Count_1,Most_common_2,Count_2,Most_common_3,Count_3,Most_common_4,Count_4,Most_common_5,...,Most_common_16,Count_16,Most_common_17,Count_17,Most_common_18,Count_18,Most_common_19,Count_19,Most_common_20,Count_20
0,Conflict,NATO,116,Verkhovna Rada,114,Yuri Boyko,76,Radovan Karadzic,73,Donald Trump,...,Bashar Assad,28,Karl Marx,27,Anatoly Hrytsenko,26,Viktor Yanukovich,26,Lech Kaczynski,25
1,Drought,Стране Басков,2,Татьяна Викторовна,2,Blair Trewin,1,Bioversity International,1,Anna Akhmatova,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,Ecology,UNESCO,10,Facebook,7,Donald Trump,5,Coca Cola,3,Verkhovna Rada,...,Билл де Блазио,2,Жизел Бюндхен,2,Rainforest Alliance,2,Carlsberg Group,2,Stephen Hawking,2
3,Genocide,Paul Kagame,1,Dassault Aviation,1,NATO,1,Петра Порошенка,1,Oleg Lyashko,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,HumanitarianAid,Nicolas Maduro,427,Verkhovna Rada,293,Donald Trump,244,NATO,189,Красного Креста,...,Петра Порошенко,88,EU,77,UNICEF,71,Vladimir Putin,65,Красный Крест,65
5,ManMadeDisasters,Volkswagen,1041,Bayerische Motoren Werke,523,Boeing,428,Opel,329,Facebook,...,Verkhovna Rada,88,Elizabeth II of the United Kingdom,83,Toyota Prius,81,Petro Poroshenko,78,Vitali Klitschko,77
6,NaturalDisasters,Facebook,37,NASA,35,Sunday Times,24,Organization for Security and Co operation in ...,19,Xinhua News Agency,...,Petro Poroshenko,12,Папуа Новая,12,Google,11,Vasily Kirilych,11,Virgin Mary,10
7,PoliticalUnrest,Verkhovna Rada,331,Petro Poroshenko,125,NATO,90,Nicolas Maduro,70,Liudmyla Denysova,...,Viktor Yanukovich,25,Organization for Security and Co operation in ...,23,The Times,22,у Порошенко,21,Нацгвардии Степану Полтораку,21
8,ScienceandTechnology,Verkhovna Rada,94,New York Times,85,Google,76,Microsoft,66,The Times,...,Washington Post,27,Microsoft Windows,25,Symantec,24,Frank Walter Steinmeier,24,Сергей Князев,22
9,Security,Facebook,364,Donald Trump,164,Organization for Security and Co operation in ...,152,NATO,142,Petro Poroshenko,...,Vladimir Pozner,35,Hamas,35,Андерса Брейвика,34,Radio Free Europe,33,Imran Khan,30


In [38]:
datab.to_csv("Ukraine_UA_Jan_01_to_Apr_06.csv", index = False)